## Import Dataset and Gemma-2b-en model.
This dataset consists of 10 idiom samples from each eligible language mentioned in the competition description, along with their literal, figurative meanings and example usage.

These Idioms have been gatherd using perplexity.ai, with prompt engineering to keep them popular and relevant to the corresponding language.

My aim with this fine-tuning approach was to create a version of Gemma2 which can understand idioms and preach them like a wise owl ;)

## Import the data into kaggle notebook

In [1]:
import pandas as pd # data processing, CSV file I/O (used here for pd.read_json)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/gemma2/keras/gemma2_2b_en/1/config.json
/kaggle/input/gemma2/keras/gemma2_2b_en/1/tokenizer.json
/kaggle/input/gemma2/keras/gemma2_2b_en/1/metadata.json
/kaggle/input/gemma2/keras/gemma2_2b_en/1/model.weights.h5
/kaggle/input/gemma2/keras/gemma2_2b_en/1/assets/tokenizer/vocabulary.spm
/kaggle/input/multilingual-idioms-indian/Gemma/burmese.json
/kaggle/input/multilingual-idioms-indian/Gemma/malayalam.json
/kaggle/input/multilingual-idioms-indian/Gemma/nepali.json
/kaggle/input/multilingual-idioms-indian/Gemma/catalan.json
/kaggle/input/multilingual-idioms-indian/Gemma/hindi.json
/kaggle/input/multilingual-idioms-indian/Gemma/croatian.json
/kaggle/input/multilingual-idioms-indian/Gemma/slovak.json
/kaggle/input/multilingual-idioms-indian/Gemma/finnish.json
/kaggle/input/multilingual-idioms-indian/Gemma/icelandic.json
/kaggle/input/multilingual-idioms-indian/Gemma/ukrainian.json
/kaggle/input/multilingual-idioms-indian/Gemma/thai.json
/kaggle/input/multilingual-idioms-indian

# Sample the dataset 


In [2]:
df= pd.read_json('/kaggle/input/multilingual-idioms-indian/Gemma/turkish.json')
df

,idiom,literal_meaning,figurative_meaning,example,language
0,Ateşle oynamak,To play with fire.,To take dangerous risks.,Investing in that company is like ateşle oynamak.,Turkish
1,Göz var nizam var,"There is an eye, there is order.",Things should be done properly.,We need to organize this event well; göz var n...,Turkish
2,Dost acı söyler,A friend speaks bitterly.,"True friends tell the truth, even if it's harsh.",He told me the truth about my performance; dos...,Turkish
3,Sakla samanı gelir zamanı,Save the straw; its time will come.,Everything has its purpose and time.,You never know when you might need it; sakla s...,Turkish
4,"Bir elin nesi var, iki elin sesi var",What does one hand have? Two hands have a voice.,Teamwork achieves more than individual effort.,Together we can achieve great things; bir elin...,Turkish
5,Gülü seven dikenine katlanır,He who loves roses must endure its thorns.,Love comes with challenges.,"If you want to be in a relationship, remember:...",Turkish
6,Ayağını yorganına göre uzat,Stretch your leg according to your blanket.,Live within your means.,Don’t spend too much money; ayağını yorganına ...,Turkish
7,Damlaya damlaya göl olur,"Drop by drop, a lake forms.",Small efforts accumulate to create something s...,Keep saving money; damlaya damlaya göl olur.,Turkish
8,Kervan yolda düzülür,The caravan is arranged on the road.,Plans can be adjusted as you go.,We’ll figure it out along the way; kervan yold...,Turkish
9,"Söz gümüşse, sükût altındır","If speech is silver, silence is golden.",Sometimes it’s better to remain silent.,"In some situations, söz gümüşse, sükût altındır.",Turkish


# Set up of Environment before loading Gemma Model

### Explanation for Environment Setup

- **`keras-nlp`**: Provides tools for NLP tasks, including tokenizers, pre-trained models, and samplers like `TopKSampler`.
- **`keras`**: Deep learning library used to build and train models, with the latest version ensuring compatibility with new APIs.

In [3]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [4]:
#Configured our environment to use JAX as the backend for Keras
#and to maximize GPU memory allocation for XLA.
import os
os.environ['KERAS_BACKEND'] = 'jax'
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [5]:
import keras
import keras_nlp

# Load the dataset

In [6]:
#Use load_dataset to generate a combined training dataset of all language files
from datasets import load_dataset

ds = load_dataset("json",data_files='/kaggle/input/multilingual-idioms-indian/Gemma/*.json')

Resolving data files:   0%|          | 0/71 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
##Display dataset summary
ds

DatasetDict({
    train: Dataset({
        features: ['idiom', 'literal_meaning', 'figurative_meaning', 'example', 'language'],
        num_rows: 720
    })
})

In [8]:
from datasets import DatasetDict
# Initialize an empty list to store the formatted examples
data = []
# Access the 'train' split of your dataset
train_data = ds["train"]

# Add debug printing to see what fields are available
print("First example keys:", list(train_data[0].keys()))

# Iterate over each example in the dataset
for i, example in enumerate(train_data):
    try:
        # Check if required fields are available and valid
        required_fields = ["idiom", "literal_meaning", "figurative_meaning", "example"]
        
        # Print missing fields for debugging
        missing_fields = [field for field in required_fields if field not in example]
        if missing_fields:
            print(f"Example {i} is missing fields: {missing_fields}")
            continue
            
        #Template with instruction and response format
        template = (
        "Instruction:\n"
        "Find a suitable idiom for this situation: {figurative_meaning}\n\n"
        "Response:\n"
        "Idiom : {idiom}\n\n"
        "Literal Meaning:{literal_meaning}\n"
        "Example Use: {example}\n"
        "Cultural Context: This idiom comes from the {language} culture.\n"
        )
        
        # Format the example and add it to the data list
        formatted_example = template.format(**example)
        data.append(formatted_example)
        
    except KeyError as e:
        print(f"KeyError in example {i}: {str(e)}")
        print(f"Available keys: {list(example.keys())}")
        continue

# Limit to the first 1000 examples
data = data[:1000]

# Display some random row of examples, to ensure the data is captured correctly.
for i, example in enumerate(data[4:10]):
    print(f"Example {i + 1}:\n{example}\n")

First example keys: ['idiom', 'literal_meaning', 'figurative_meaning', 'example', 'language']
Example 1:
Instruction:
Find a suitable idiom for this situation: Sometimes it's better to remain silent than to speak.

Response:
Idiom : إذا كان الكلام من فضة فالسكوت من ذهب

Literal Meaning:If speech is silver, silence is golden.
Example Use: In this situation, remember: إذا كان الكلام من فضة فالسكوت من ذهب.
Cultural Context: This idiom comes from the Arabic culture.


Example 2:
Instruction:
Find a suitable idiom for this situation: Don't harm what you might need later.

Response:
Idiom : لا تبصق في البئر

Literal Meaning:Don't spit in the well.
Example Use: He always remembers not to spit in the well; you never know when you’ll need it.
Cultural Context: This idiom comes from the Arabic culture.


Example 3:
Instruction:
Find a suitable idiom for this situation: Hard work leads to success.

Response:
Idiom : من جد وجد

Literal Meaning:He who strives will find.
Example Use: He believes tha

# Load Model

In [9]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

# Enable LoRA for the model
The trainable parameters, 2,614,341,888 are very large (9.74 GB) to train on our limited GPU, so to be able to actually train these on our systems, we need to use Lower Order Rank Adaptation(LORA). 

Basically converting 16bit floating point weights and parameters to 4bit.
Of course there is some amount of loss in this approach, but the tradeoff for functionality is important to us.

In [10]:
# Enable LoRA for the model and set the LoRA rank to 4 (for 4 bit)
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

As you can see, the size of trainable parameters have reduces to about 11.17 MB. Now we can train the model on our dataset.

In [11]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=2, batch_size=1)

Epoch 1/2
720/720 ━━━━━━━━━━━━━━━━━━━━ 346s 443ms/step - loss: 0.5953 - sparse_categorical_accuracy: 0.6324
Epoch 2/2
720/720 ━━━━━━━━━━━━━━━━━━━━ 320s 419ms/step - loss: 0.3285 - sparse_categorical_accuracy: 0.7899


# Testing it with different Prompts

In [12]:
# list of meanings for you to try from
test_meanings = [
    "feeling stuck in a difficult situation",
    "making a big decision",
    "working hard to achieve a goal",
    "dealing with unexpected problems",
    "celebrating a success",
    "overcoming failure",
    "dealing with a challenging coworker",
    "handling stress at work",
    "feeling overwhelmed with responsibilities",
    "facing a tough deadline",
    "getting lost in a new city",
    "managing work-life balance",
    "feeling homesick in a new place",
    "learning something new and challenging",
    "handling a sudden change in plans"
]

### Testing prarameters

- **Top-K Sampling** (`keras_nlp.samplers.TopKSampler(k=10, seed=2)`):
  - Limits token choices to the top 10 most probable at each generation step (`k=10`).
  - `seed=2` ensures reproducibility of the result for every time it is executed.

- **Compiling Model**:
  - `gemma_lm.compile(sampler=sampler)` To set the sampling method for text generation.

- **Text Generation**:
  - `gemma_lm.generate(prompt, max_length=256)` To generate text from the `prompt` with a maximum length of 256 tokens using Top-K sampling.

I used this approach because it balances randomness and coherence in the generated text.

**Example 1**

In [13]:
test_meaning = "facing a tough deadline"

#Using the same template format as training
prompt = (
    "Instruction:\n Do you know any idiom which would be suitable for {}\n\n"
    "Response:\n"
).format(test_meaning)

sampler = keras_nlp.samplers.TopKSampler(k=10, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
 Do you know any idiom which would be suitable for facing a tough deadline

Response:
Idiom: 99问题中有1个答案

Literal Meaning:In 99 problems, there is one solution.
Example Use: In every situation, always remember:99问题中有1个答案.
Cultural Context: This idiom comes from the Chinese culture.



**Example 2**

In [14]:

#You can adjust these paramters to test them on your own 
test_meaning = "someone who does not value another person"

#Using the same template format as training
prompt = (
    "Instruction:\n Do you know any idiom which would be suitable for {}\n\n"
    "Response:\n"
).format(test_meaning)

sampler = keras_nlp.samplers.TopKSampler(k=10, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
 Do you know any idiom which would be suitable for someone who does not value another person

Response:
Idiom: 錢不是親

Literal Meaning:Money is not a relative.
Example Use: Don't let your success go to your head. 錢不是親; success is not close family.
Cultural Context: This idiom comes from the Taiwanese culture.



**Example 3**

In [15]:
#3
test_meaning = "managing work-life balance"

prompt = (
    "Instruction:\n Do you know any idiom which would be suitable for {}\n\n"
    "Response:\n"
).format(test_meaning)

sampler = keras_nlp.samplers.TopKSampler(k=10, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
 Do you know any idiom which would be suitable for managing work-life balance

Response:
Idiom: 40% of work, 60% life

Literal Meaning:Forty percent of work and sixty percent of life.
Example Use: I think it's important to focus on both work and family equally; 40% of work and 60% of life.
Cultural Context: This idiom comes from the Chinese culture.

